# CrewAI In Depth

## Traditional App Development vs. LLM and Multi-Agent App Development

#### Traditional App Development (Example: any traditional software application)
* Clearly defined input formats: string, integer, etc.
* Clearly defined transformations to apply to the inputs: math calculation, loops, etc.
* Clearly defined output formats: string, integer, etc.
* If you run the program again, for the same input you will have always the same output.

#### LLM and Multi-Agent App Development (Example: ChatGPT)
* Fuzzy inputs: open-ended text, it can be different types of text (string, markdown, tabular data, math operation, etc). You don't know what the user is going to enter as input.
* Fuzzy transformations: You don't know if the LLM will transform the input into a list, write a paragraph, answer a question, brainstorm new ideas, perform logic reasoning, math reasoning, etc.
* Fuzzy output: open-ended text, it can be different types of text (paragraph, numbers, json, markdown, etc).
* If you run the program again, for the same input you will NOT have always the same output.

## Key elements of Multi-Agent Apps

#### Role Playing
* A good role definition can make a huge difference on the responses you are going to get from agents.

#### Focus
* Multiple specialized agents have better results than one know-it-all agent.

#### Tools
* Do not overload your agents with too many tools.
* Choose the tools carefully.

#### Cooperation
* Take feedback.
* Delegate tasks.

#### Guardrails
* To avoid:
    * Hallucinations.
    * Infinite loops.
    * Etc.
* To enforce:
    * Steps.
    * Output format.
    * Etc.

#### Memory
* Memory is the factor that can make a bigger impact in the performance of your agents.
* Memory = ability for the agent to remember what it has done previously and to use that to inform new decisions and new executions.
* In CrewAI you have 3 types of memory for free, out of the box:
    *  Short-term memory:
        *  Lives only during the Crew execution and
        *  It is shared accross all Agents of the Crew.
    *  Long-term memory:
        *  Lives even after the Crew finishes.
        *  Stored in a database locally.
        *  Allows the Agents to learn from previous executions.
        *  Leads to "self-improving" agents.
    *  Entity memory.
        *  Lives only during the Crew execution.
        *  Stores the subjects that are being discussed: people names, company names, locations, etc.

## How do multi-agents collaborate? "Processes" define how agents collaborate.
* Sequentially: one task after the other.
* Hierarchical: one manager and one team.
    * The manager always remember the initial goal.
    * The manager delegates.
    * The manager reviews and can ask for further improvements. 
* In parallel.
    * Asyncronously. 

## How can it be delegation among Agents?
* Delegation: agents ask questions to each other.
    * This can happen in any kind of process: sequential, hierarchical and in parallel.

## Exception Handling in CrewAI
* By default, CrewAI does not stop the app when it finds an error. Instead, the Crew tries to use an alternative way.

## Multi-Agent App Setup with CrewAI

#### Recommended: create new virtualenv
* pyenv virtualenv 3.11.4 your_venv_name
* pyenv activate your_venv_name
* pip install jupyterlab
* jupyter lab

* Install:
    * crewai
    * crewai_tools
    * langchain_community 

In [1]:
#!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

* Import warnings package to ignore warnings:

In [3]:
import warnings
warnings.filterwarnings('ignore')

## .env file

#### .env File
Remember to include:
OPENAI_API_KEY=your_openai_api_key

LANGCHAIN_TRACING_V2=true
LANGCHAIN_ENDPOINT=https://api.smith.langchain.com
LANGCHAIN_API_KEY=your_langchain_api_key
LANGCHAIN_PROJECT=your_project_name

## Basic CrewAI imports: Agent, Task and Crew modules

In [6]:
from crewai import Agent, Task, Crew

## LLM Selection

* The best LLM to use with CrewAI is ChatGPT-4.
* Remember, ChatGPT-4 is the most expensive model of OpenAI.
* For this demo we are going to choose a different LLM, the more convenient ChatGPT-3.5 version.
* We need to make sure to get our OpenAI API Key from the .env file:

In [7]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [8]:
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

* To try it with ChatGPT-4, comment out the previous cell and uncomment the following one:

In [9]:
# os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'

## Multi-Agent App Creation process with CrewAI
* App planning.
* Import pre-built tools and create custom tools.
* Create the Agents.
* Create the Tasks.
* Create the Crew.
* Run the Crew with the input.
* Optional: display output in the notebook.

## App Planning
#### Mental Framework for Agent Creation
* Think like a manager.
    * What is the goal?
    * What is necessary process to follow in order to reach the goal?
    * What kind of people I would hire to get this done?
        * roles.
        * goals.
        * backstories.
* Be as specific as possible when you define Agents and Tasks.

#### Define goal. Examples:
* Research and write an article about a topic.
* Provide customer support about a product.
* Create a customer outreach campaign.
* Plan an event.
* Financial Analysis.
* Tailor job applications.
#### Define use case. Example:
* Analyze a job offer.
* Analyze a candidate resume.
* Customize the resume for the offer.
* Prepare talking points for job interview.
#### Define input. Example:
* Python dictionary:
    * Resume of the candidate
    * URL of the job offer
    * GitHub profile of the candidate
    * Intro of the candidate
* **Important: you will interpolate input items in Agent and Task definitions**
#### Define output. Example:
* Customized resume.
* Talking points for job interview.

## Import tools

#### What makes a great tool?
* Versatile.
    * Able to accept fuzzy inputs and outputs. 
* Fault-tolerant.
    * Do not stop execution. Fail gracefully, send the error message back to the agent, and try again. 
* Implement Smart Caching.
    * Having a caching layer that prevents innecessary requests is crucial.
    * **CrewAI offers Cross-Agent Caching**: if one Agent tries to use a tool with a given set of arguments, and another agent tries to use the same tool with the same set of arguments -even if they are different Agents- they are going to use a caching layer so the second time they use the tool they are not going to make the API call. That:
        * Prevents unnecessary requests.
        * Prevents hitting rate limits (number of requests per second allowed for an API call).
        * Saves execution time. Retrieving cached results is much faster than an API call.

#### Import Pre-Built Tools
* CrewAI pre-built tools.
* LangChain pre-built tools.

#### Examples of Pre-Built Tools
* SerperDevTool: google search (requires API Key)
* ScrapeWebsiteTool: scrape content from URL.
* WebsiteSearchTool: RAG over a website.

In [12]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

* We can configure the tool so it will only be used to scrape content from this specific URL:

In [24]:
# docs_scrape_tool = ScrapeWebsiteTool(
#     website_url="https://aiaccelera.com/ai-consulting-for-businesses/"
# )

In [39]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool

In [25]:
# directory_read_tool = DirectoryReadTool(directory='./my_directory')
# file_read_tool = FileReadTool()
# search_tool = SerperDevTool()

* MDXSearchTool: RAG over a file.

In [27]:
# from crewai_tools import MDXSearchTool

# semantic_search_document = MDXSearchTool(mdx='./my_document.md')

#### Create Custom Tools

- Every Tool needs to have a `name` and a `description`.
- You can customize the code with your logic in the `_run` function.

In [15]:
from crewai_tools import BaseTool

class EmotionAnalysisTool(BaseTool):
    name: str = "Emotion Analysis Tool"
    description: str = ("Examines the emotional tone of written content "
         "to promote positive and captivating interactions.")
    
    def _run(self, text: str) -> str:
        return "positive"


In [28]:
# emotion_analysis_tool = EmotionAnalysisTool()

#### How to assign tools
* We can assign tools at:
    * Agent Level: the Agent can use the tool in any task.
    * or at Task Level: the Agent can only use the tool when performing this particular task. Task tools override Agent tools.

## Create Agents

Creating an Agent in CrewAI is very simple using the Agent module. In order to define the agent, we need to specify:
* Role: role name.
* Goal.
* Backstory: role definition.

In [35]:
# content_coordinator = Agent(
#     role="Content Coordinator",
#     goal="Develop captivating and precise content about {topic}",
#     backstory="You are engaged in preparing a blog post "
#               "concerning the subject: {topic}. "
#               "You gather data that assists the "
#               "audience in acquiring knowledge "
#               "and making educated choices. "
#               "Your efforts lay the groundwork for "
#               "the Content Writer to create a detailed piece on this topic.",
#     allow_delegation=False,
#     verbose=True
# )


* Note: using multiple strings instead of the triple quote docsctring `"""` we avoid whitespaces and newline characters and that way our backstory is better formatted to pass it to the LLM.
* As you can see, we are interpolating the variable `{topic}`. This variable would come from the input.
* `allow_delegation=False`: this agent is not allowed to delegate tasks to other agent.
* Note: by default, `allow_delegation` is set to True.
* `verbose=True`: display log of events and thoughts of the agent.

* Agents can use tools:

In [ ]:
# location_manager = Agent(
#     role="Location Manager",
#     goal="Locate and reserve a suitable venue "
#     "in accordance with the specifics of the event",
#     tools=[search_tool, scrape_tool],
#     verbose=True,
#     backstory=(
#         "With a sharp eye for space and "
#         "a grasp of event coordination, "
#         "you specialize in identifying and booking "
#         "the ideal location that aligns with the event's motif, "
#         "capacity, and financial limits."
#     )
# )

## Create Tasks

#### Key elements of well defined tasks
* Clear description of the task.
* Clear and concise expectation.

Creating a Task in CrewAI is very simple using the Task module. In order to define the task, we need to specify:
* Task Description.
* Expected Output.
* Agent that will perform the task.

In [34]:
# content_planning_task = Task(
#     description=(
#         "1. Focus on the most current trends, influential figures, "
#             "and significant updates on {topic}.\n"
#         "2. Define the intended audience, taking into account "
#             "their interests and concerns.\n"
#         "3. Create a thorough content framework that includes "
#             "an opening, main points, and a call to engage.\n"
#         "4. Incorporate relevant SEO terms and necessary data or references."
#     ),
#     expected_output="An extensive content strategy document "
#         "with a framework, analysis of the audience, "
#         "SEO terms, and references.",
#     agent=planner,
# )

* Tasks can include tools:

In [32]:
# customized_outreach_task = Task(
#     description=(
#         "Leverage the insights from "
#         "the lead profiling analysis on {lead_name}, "
#         "to devise a targeted outreach initiative "
#         "directed at {key_decision_maker}, "
#         "the {position} at {lead_name}. "
#         "This initiative should highlight their latest {milestone} "
#         "and how our solutions can aid their objectives. "
#         "Your messaging should align "
#         "with {lead_name}'s organizational culture and ethics, "
#         "showing a profound grasp of "
#         "their industry and requirements.\n"
#         "Avoid presumptions and strictly "
#         "rely on verified information."
#     ),
#     expected_output=(
#         "A sequence of customized email templates "
#         "designed for {lead_name}, "
#         "specifically aimed at {key_decision_maker}. "
#         "Each template should weave "
#         "an engaging story that ties our solutions "
#         "to their recent successes and aspirations. "
#         "Ensure the tone is captivating, formal, "
#         "and consistent with {lead_name}'s business ethos."
#     ),
#     tools=[emotion_analysis_tool, search_tool],
#     agent=lead_sales_rep,
# )

#### Hyperparameters you can use with CrewAI when you define a Task
* Set a context.
* Set a callback.
* Override Agent tools with Task tools.
* Force human input before end of task.
* Execute asynchronously.
* Output as Pydantic Object.
* Output as JSON Object.
* Output as a file.
* Run in parallel.

#### Creating Location Pydantic Object

- Create a class `LocationDetails` using [Pydantic BaseModel](https://docs.pydantic.dev/latest/api/base_model/).
- Agents will populate this object with information about different locations by creating different instances of it.

In [40]:
from pydantic import BaseModel
# Define a Pydantic model for location details 
# (demonstrating Output as Pydantic)
class LocationDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

- By using `output_json`, you can specify the structure of the output you want. In this case, the Pydantic model.
- By using `output_file`, you can get your output in a file.
- By setting `human_input=True`, the task will ask for human feedback (whether you like the results or not) before finalising it.

In [31]:
# location_task = Task(
#     description="Locate a facility in {event_city} "
#                 "that fulfills the requirements for {event_topic}.",
#     expected_output="Complete information about a selected "
#                     "facility you identified to host the event.",
#     human_input=True,
#     output_json=LocationDetails,
#     output_file="location_details.json",  
#     agent=location_coordinator
# )


- By setting `async_execution=True`, it means the task can run in parallel with the tasks which come after it.

In [ ]:
# catering_task = Task(
#     description="Arrange catering and "
#                  "equipment for a gathering "
#                  "with {expected_participants} attendees "
#                  "on {tentative_date}.",
#     expected_output="Verification of all logistics preparations "
#                     "including food service and equipment arrangement.",
#     human_input=True,
#     async_execution=True,
#     agent=catering_manager
# )


- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.
- The following task will output a file called `tailored_resume.md`

In [23]:
# resume_customization_task = Task(
#     description=(
#         "Utilize the profile and job specifications collected from "
#         "prior tasks to customize the resume, emphasizing the most "
#         "pertinent sections. Use tools to refine and improve the "
#         "resume content. Ensure this is the most effective resume possible but "
#         "refrain from fabricating any details. Revise every segment, "
#         "including the opening summary, employment history, skills, "
#         "and education sections, all to better reflect the candidate's "
#         "capabilities and alignment with the job description."
#     ),
#     expected_output=(
#         "A revised resume that adeptly showcases the candidate's "
#         "skills and experiences pertinent to the job."
#     ),
#     output_file="customized_resume.md",
#     context=[research_task, profile_task],
#     agent=resume_strategist
# )


## Create Crew

* By default, in CrewAI the tasks will be performed sequentially, so the order of the task in the list matters.
    * The output of task1 is going to be sent as part of the input of task2. 

In [ ]:
# crew = Crew(
#     agents=[planner, writer, editor],
#     tasks=[plan, write, edit],
#     verbose=2
# )

* `memory=True` enables all the 3 memory types for this Crew (short-term, long-term and entity memories).

In [36]:
# crew = Crew(
#   agents=[support_agent, quality_assurance_agent],
#   tasks=[inquiry_resolution, quality_assurance_review],
#   verbose=2,
#   memory=True
# )

* If you set `async_execution=True` for several tasks, the order for them will not matter in the Crew's tasks list.

- If you set `human_input=True` for some tasks, the execution of the Crew will ask for your input before it finishes running.
- When it asks for feedback, use your mouse pointer to first click in the text box before typing anything.

- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use. This manager will delegate in the Agents of the Crew to perform their tasks.
- In future versions of CrewAI you will be able to set the Manager Agent yourself. By now, you can select the LLM who will act as the manager agent and CrewAI will create this agent internally for you.
- See that the Crew Manager kickoffs the Crew.

In [37]:
# from crewai import Crew, Process
# from langchain_openai import ChatOpenAI

# # Define the crew with agents and tasks
# financial_analysis_crew = Crew(
#     agents=[data_analyst, 
#             trading_strategist, 
#             execution_agent, 
#             risk_management_exec],
    
#     tasks=[data_analysis_task, 
#            trading_strategy_task, 
#            execution_task, 
#            risk_management_task],
    
#     manager_llm=ChatOpenAI(model="gpt-4-turbo", 
#                            temperature=0.7),
#     process=Process.hierarchical,
#     verbose=True
# )

## Run Crew with the Input

In [38]:
# inputs = {
#     "customer": "AI Accelera",
#     "person": "Julio Colomer",
#     "inquiry": "I need help with setting up a Multi-Agent App "
#                "and kicking it off, specifically "
#                "how can I add memory? "
#                "Can you provide guidance?"
# }
# result = crew.kickoff(inputs=inputs)

## Optional: Display results in Notebook

- Display the generated `file_name.md` file.

**Note**: After `kickoff` execution has successfully ran, wait an extra 45 seconds for the `file_name.md` file to be generated. If you try to run the code below before the file has been generated, your output would look like:

```
file_name.md
```

If you see this output, wait some more and than try again.

In [ ]:
# from IPython.display import Markdown

# Markdown(result)

In [ ]:
# import json
# from pprint import pprint

# with open('venue_details.json') as f:
#    data = json.load(f)

# pprint(data)

## Interesting Links to learn more
* **crewai.com** to learn more about CrewAI.
    * Documentation.
    * How To Guides.
    * Chat with the documentation.
    * Enterprise solutions.
        * In beta.
        * Looking for early adopters: waiting list.
        * Turn any Crew into an API within seconds.
        * Connect to your apps using hooks, REST, gRPC and more.
        * Get access to templates, customo tools and early UI.
        * Get business support, SLA, private VTC.
* **CrewAI Plus** (in beta, by invitation only).
    * Deploy your Crews from GitHub.
    * Transform your Crews in APIs in a matter of minutes.
        * Enter your private variables.
        * Have the URL of your API with SSL, hosted in a private VTC, outscaling, everything that makes it ready for a production use case.
    * Sidebar:
        * Crews.
        * Templates.
        * Dashboad.
        * UI Studio.
        * Storage.
        * Connectors.